In [1]:
import torch
import nn

In [2]:
err_inp = torch.jit.load("../badinput.tensor").state_dict()["0"]

amdgpu.ids: No such file or directory


In [3]:
s = 8
err_inp[0, s*14:s*14+14, :, :]

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

        [[72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.],
         [72., 72., 72., 72., 72., 72., 72., 72.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0

In [4]:
err_inp[0, 112:119, 0, 0]

tensor([ 0., 72.,  0.,  0.,  0.,  0., 26.], device='cuda:0')

In [5]:
model_name = "../runs/016/tb_logs/chess/version_4/epoch:8-2.907-0.314"
model_ckpt = model_name + ".ckpt"
model_pt = model_name + ".pt"

In [6]:
pt = torch.jit.load(model_pt)

orig_module = nn.ChessModule19()
orig_module.load_state_dict(torch.load(model_ckpt, weights_only=True), strict=True)
orig_module = orig_module.eval()
orig_module.cuda()

inuse_module = nn.load_model(checkpoint=model_ckpt, inference=True, compile=False, device="cuda")

..loading checkpoint:  ../runs/016/tb_logs/chess/version_4/epoch:8-2.907-0.314.ckpt


In [7]:
with torch.no_grad():
    o = orig_module(err_inp)
o

(tensor([[-11.9228, -20.6833, -21.3826,  ..., -17.9548, -17.9068, -17.8975]],
        device='cuda:0'),
 tensor([[0.0063]], device='cuda:0'))

In [8]:
with torch.no_grad():
    o = inuse_module(err_inp)
o

(tensor([[-11.9228, -20.6833, -21.3826,  ..., -17.9548, -17.9068, -17.8975]],
        device='cuda:0'),
 tensor([[0.0063]], device='cuda:0'))

In [10]:
pt(err_inp.bfloat16())

(tensor([[-12.0000, -20.7500, -21.3750,  ..., -18.0000, -18.0000, -18.0000]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[0.0063]], device='cuda:0', dtype=torch.bfloat16))

----

In [11]:
x = torch.randn(1, 119, 8, 8, dtype=torch.float32).cuda()
with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.float16, cache_enabled=False):
        orig_module_jit = torch.jit.trace(orig_module, [x])

In [12]:
orig_module_jit(err_inp)

(tensor([[-11.9156, -20.6812, -21.3843,  ..., -17.9546, -17.9077, -17.8999]],
        device='cuda:0', grad_fn=<LogSoftmaxBackward0>),
 tensor([[0.0063]], device='cuda:0', dtype=torch.float16,
        grad_fn=<TanhBackward0>))

In [13]:
orig_module_script = torch.jit.script(orig_module)

In [14]:
with torch.no_grad():
    with torch.autocast(device_type="cuda"):
        o = orig_module_script(err_inp)
o

(tensor([[-11.9230, -20.6887, -21.3762,  ..., -17.9543, -17.9074, -17.8996]],
        device='cuda:0'),
 tensor([[0.0063]], device='cuda:0', dtype=torch.float16))

In [15]:
with torch.no_grad():
    with torch.autocast(device_type="cuda"):
        o = orig_module_script.conv_block(err_inp)
        for i in range(10):
            o = orig_module_script.res_blocks[i](o)
        o2 = orig_module_script.res_blocks[10](o)
o.mean(), o2.mean()

(tensor(0.2235, device='cuda:0', dtype=torch.float16),
 tensor(0.2473, device='cuda:0', dtype=torch.float16))

In [16]:
orig_module_script = torch.jit.script(orig_module.bfloat16())
orig_module_script = torch.jit.optimize_for_inference(orig_module_script)
with torch.no_grad():
    o = orig_module_script(err_inp.bfloat16())
o

(tensor([[-12.0000, -20.7500, -21.3750,  ..., -18.0000, -18.0000, -18.0000]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[0.0063]], device='cuda:0', dtype=torch.bfloat16))

In [17]:
orig_module_script.res_blocks[0]

AttributeError: 'RecursiveScriptModule' object has no attribute 'res_blocks'

In [139]:
mm = torch.jit.script(orig_module.bfloat16())
mm = torch.compile(mm, mode="reduce-overhead", fullgraph=True)
with torch.no_grad():
    o = mm(err_inp.bfloat16())
o

(tensor([[-11.2500, -15.2500, -14.0000,  ..., -17.8750, -18.1250, -18.0000]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[-1.]], device='cuda:0', dtype=torch.bfloat16))

In [20]:
t = err_inp.bfloat16()

In [140]:
%timeit with torch.no_grad(): mm(t)

3.9 ms ± 21.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [141]:
%timeit with torch.no_grad(): orig_module_script(t)

2.69 ms ± 56.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
pt11 = torch.jit.load("../runs/000/last-bf16-l11.pt")
pt13 = torch.jit.load("../runs/000/last-bf16-l13.pt")
pt19 = torch.jit.load("../runs/000/last-bf16-l19.pt")

In [35]:
%timeit with torch.no_grad(): pt11(t)

1.81 ms ± 74.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [36]:
%timeit with torch.no_grad(): pt13(t)

2.17 ms ± 42 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
%timeit with torch.no_grad(): pt19(t)

3.11 ms ± 69.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
